<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Baseline搭建" data-toc-modified-id="Baseline搭建-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Baseline搭建</a></span><ul class="toc-item"><li><span><a href="#离散小波变换" data-toc-modified-id="离散小波变换-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>离散小波变换</a></span></li><li><span><a href="#数据读取" data-toc-modified-id="数据读取-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>数据读取</a></span></li><li><span><a href="#网络搭建与可视化" data-toc-modified-id="网络搭建与可视化-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>网络搭建与可视化</a></span></li><li><span><a href="#网络搭建与可视化-原版网络" data-toc-modified-id="网络搭建与可视化-原版网络-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>网络搭建与可视化-原版网络</a></span></li><li><span><a href="#3.4-模型配置与训练" data-toc-modified-id="3.4-模型配置与训练-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>3.4 模型配置与训练</a></span></li><li><span><a href="#模型预测与结果提交" data-toc-modified-id="模型预测与结果提交-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>模型预测与结果提交</a></span></li></ul></li><li><span><a href="#总结" data-toc-modified-id="总结-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>总结</a></span></li><li><span><a href="#以下为草稿" data-toc-modified-id="以下为草稿-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>以下为草稿</a></span></li></ul></div>

In [1]:
!pip install kaggle
!pip install pywavelets
!pip install visualdl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Baseline搭建




![](https://ai-studio-static-online.cdn.bcebos.com/b4fc0d27a6b44110a185b564c51a59ac7901f976232e4ac2bba9308875b81780)


&emsp;&emsp;在Baseline中，提出了一种双分支网络，首先將脑纹信号进行离散小波变换，将脑纹信号拆解为两个函数，分别入对应的网络通路，网络整体设计思想参考了双线性卷积神经网络的结构。对于脑纹识别问题采取了分而治之的思想。网络整体结构如上图所示。


### 离散小波变换


![](https://ai-studio-static-online.cdn.bcebos.com/4330d0b8d8174aea92f858fb326802f90a38ea95def84eac839918d824afa259)
![](https://ai-studio-static-online.cdn.bcebos.com/f1b93632edea47a39a68345ce4ea8e04aca61c97144840e7ae128025f1de025b)

<br>

&emsp;&emsp;如上图所示，脑纹信号经过离散小波变换之后可以拆分为两组不同的函数。

&emsp;&emsp;小波分解的意义就在于能够在不同尺度上对信号进行分解，而且对不同尺度的选择可以根据不同的目标来确定。对于许多信号，低频成分相当重要，它常常蕴含着信号的特征，而高频成分则给出信号的细节或差别。人的话音如果去掉高频成分，听起来与以前可能不同，但仍能知道所说的内容；如果去掉足够的低频成分，则听到的是一些没有意义的声音。在小波分析中经常用到近似与细节。近似表示信号的高尺度，即低频信息；细节表示信号的高尺度，即高频信息。因此，原始信号通过两个相互滤波器产生两个信号。


参考资料：[离散小波变换（DWT）](https://blog.csdn.net/daaikuaichuan/article/details/81387673)


In [1]:
! pip install tqdm
! sudo apt-get update
import os
from tqdm import tqdm

def install_package_with_apt(package_name):
    os.system(f"sudo apt-get install --allow-change-held-packages --yes {package_name}")

def install_package_with_pip(package_name):
    os.system(f"pip install {package_name}")

apt_packages = [
    "texlive-latex-recommended",
    "dvipng",
    "texlive-latex-extra",
    "texlive-fonts-recommended",
    "cm-super",
]

pip_packages = [
    "Pillow",
    "umap",
    "xgboost",
    "seaborn",
    "SciencePlots",
    "pytorch-tabnet",
    "pycountry",
    "us",
    "uszipcode",
    "torch transformers",
    "geopandas matplotlib pandas",
    "geopandas descartes mapclassify",
    "SciencePlots",
    "geopandas matplotlib pandas",
    "ta",
    "pytrends",
    "chinese-calendar",
    "ephem",
    "category_encoders",
    "fbprophet",
    "statsmodels",
]

for package in tqdm(apt_packages):
    install_package_with_apt(package)

for package in tqdm(pip_packages):
    install_package_with_pip(package)

^C
Traceback (most recent call last):
  File "/opt/conda/bin/pip", line 5, in <module>
    from pip._internal.cli.main import main
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 967, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 677, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 818, in get_code
  File "<frozen importlib._bootstrap_external>", line 917, in get_data
KeyboardInterrupt
0% [Working]^C


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



### 数据读取

In [1]:
import scipy.io as scio
from sklearn.utils import shuffle
import os
import pandas as pd
import numpy as np
from PIL import Image

import paddle
import paddle.nn as nn
from paddle.io import Dataset
import paddle.vision.transforms as T
import paddle.nn.functional as F
from paddle.metric import Accuracy
from sklearn.preprocessing import LabelEncoder
from paddle.optimizer.lr import LinearWarmup
from paddle.optimizer.lr import CosineAnnealingDecay
import warnings
warnings.filterwarnings("ignore")

os.chdir('/home/u200810216/jupyter/kaggle_m3cv')
# # 读取数据
# train_images = pd.read_csv('data/data151025/Enrollment_Info.csv')
# val_images = pd.read_csv('data/data151025/Calibration_Info.csv')

# 读取数据
train_images = pd.read_csv('Enrollment_Info.csv')
val_images = pd.read_csv('Calibration_Info.csv')


train_images = shuffle(train_images, random_state=0)
val_images = shuffle(val_images)
# 划分训练集和校验集

train_image_list = train_images
val_image_list = val_images

df = train_image_list
train_image_path_list = train_image_list['EpochID'].values
train_label_list = train_image_list['subject'].values


val_image_path_list = val_image_list['EpochID'].values
val_label_list = val_image_list['SubjectID'].values

上面可以看出是经过打乱了的，至此上述代码都是看懂了的，也没用到paddlepaddle

<!-- 下面是一些关于zip语句的例子
>>> x = [1, 2, 3]
>>> y = [4, 5, 6]
>>> zipped = zip(x, y)
>>> list(zipped)
[(1, 4), (2, 5), (3, 6)]
>>> x2, y2 = zip(*zip(x, y))
>>> x == list(x2) and y == list(y2)
True -->

In [2]:
# 定义数据读取类
class MyDataset(paddle.io.Dataset):
    """
    步骤一：继承paddle.io.Dataset类
    """
    def __init__(self, train_img_list, val_img_list,train_label_list,val_label_list, mode='train'):
        """
        步骤二：实现构造函数，定义数据读取方式，划分训练和测试数据集
        """
        super(MyDataset, self).__init__()
        self.img = []
        self.label = []
        # 借助pandas读csv的库
        self.train_images = train_img_list
        self.test_images = val_img_list
        self.train_label = train_label_list
        self.test_label = val_label_list
        if mode == 'train':
            # 读train_images的数据
            for img,la in zip(self.train_images, self.train_label):
                self.img.append('/home/u200810216/jupyter/kaggle_m3cv/train/'+img+'.mat')
                # img是一个列表用来存放训练集的每一个脑纹文件的位置
                self.label.append(paddle.to_tensor(int(la[4:]) - 1, dtype='int64'))
                # la就是label，label就是subjectid 类如sub001 这样取出来的应该就是00 01 02之类的，减一是应该为了形式上的方便。
        else:
            # 读test_images的数据
            for img,la in zip(self.test_images, self.test_label):
                self.img.append('/home/u200810216/jupyter/kaggle_m3cv/val/'+img+'.mat')
                self.label.append(paddle.to_tensor(int(la[4:]) - 1, dtype='int64'))

    def load_eeg(self, eeg_path):
        
        data = scio.loadmat(eeg_path)
        return data['epoch_data']

    def __getitem__(self, index):
        """
        步骤三：实现__getitem__方法，定义指定index时如何获取数据，并返回单条数据（训练数据，对应的标签）
        """
        eeg_data = self.load_eeg(self.img[index])
        eeg_label = self.label[index]
        # label = paddle.to_tensor(label)
        
        return eeg_data,eeg_label

    def __len__(self):
        """
        步骤四：实现__len__方法，返回数据集总数目
        """
        return len(self.img)

In [3]:
#train_loader
num_workers = 0
train_dataset = MyDataset(train_img_list=train_image_path_list, val_img_list=val_image_path_list, train_label_list=train_label_list, val_label_list=val_label_list, mode='train')
train_loader = paddle.io.DataLoader(train_dataset, places=paddle.CUDAPlace(0), batch_size=256, shuffle=True, num_workers= num_workers)
# 感觉这里定义这个MyDataset类的原因是为了符合paddle的格式

#val_loader
val_dataset = MyDataset(train_img_list=train_image_path_list, val_img_list=val_image_path_list, train_label_list=train_label_list, val_label_list=val_label_list, mode='test')
val_loader = paddle.io.DataLoader(val_dataset, places=paddle.CUDAPlace(0),  batch_size=256, shuffle=True, num_workers= num_workers)


### 网络搭建与可视化

In [ ]:
import math
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
import pywt
from paddle.nn import Linear, Dropout, ReLU
from paddle.nn import Conv1D, MaxPool1D
from paddle.nn.initializer import Uniform
from paddle.fluid.param_attr import ParamAttr
from paddle.utils.download import get_weights_path_from_url

class MyNet_dwt(nn.Layer):# dwt是离散小波变换的意思？？？

    def __init__(self, num_classes=1000):#这个1000在这里是一个摆设，实际上用的是95
        super(MyNet_dwt, self).__init__()
        self.num_classes = num_classes
        
        self._conv1 = Conv1D(
            65,
            128,# 这样来看，65通道 每行1000个元素，一维的padding为1后，再stride为2，得到每行1001个元素，filter后500个元素
                # filter的初始权重不同而造成filter数量不受什么65 的限制
                # filter的实际大小应该是65*3的大小
            3,
            stride=2,
            padding=1,
            )      

        self._conv2_1 = Conv1D(
            128,
            256,
            3,
            stride=2,
            padding=1,
            )  
        
        self._conv3_1 = Conv1D(
            256,
            512,
            3,
            stride=2,
            padding=1,
            )
        self._conv4_1 = Conv1D(
            512,
            1024,
            3,
            stride=2,
            padding=1,
            )
        
        self._conv5_1 = Conv1D(
            1024,
            512,
            3,
            stride=2,
            padding=1,
            )
        self._conv6_1 = Conv1D(
            512,
            256,
            3,
            stride=2,
            padding=1,
            )        
        
        self._conv2_2 = Conv1D(
            128,
            256,
            3,
            stride=2,
            padding=1,
            )  
        self._conv3_2 = Conv1D(
            256,
            512,
            3,
            stride=2,
            padding=1,
            )
        
        self._conv4_2 = Conv1D(
            512,
            1024,
            3,
            stride=2,
            padding=1,
            )
        self._conv5_2 = Conv1D(
            1024,
            512,
            3,
            stride=2,
            padding=1,
            )
        self._conv6_2 = Conv1D(
            512,
            256,
            3,
            stride=2,
            padding=1,
            )   

        self._fc8 = Linear(
            in_features=4096,
            out_features=num_classes,
            )

    def forward(self, inputs):
        # 这个函数应该是在这里定义那些层与层之间的具体连接方式
        # 不是说上一个层直接输入到下一个层的，他可能自己在层与层之间加一些东西。
#         print("conv1层原始输入：",inputs)
        x = self._conv1(inputs)
#         print("conv1层原始输出：",x)
        x = paddle.to_tensor(pywt.dwt(x.numpy(), 'haar'), dtype='float32')
#         print("DWT处理后：",x)
        # 128行个500长度的行，经过变换变成256个250长度的行（这256个中两两配对）
        x1,x2 = x.split(2)
#         print("x1：",x1,"x2:",x2)
        # 现在又变回128个行与128个行，分别代表不同种信息。
        x1 = x1.squeeze(axis=0)
        # x1应该是1*512*128*250的tensor，他现在变成512*128*250的了
#         print("x1_squeezed:",x1)
        x2 = x2.squeeze(axis=0)
        x1 = self._conv2_1(x1)
#         self._conv2_1 = Conv1D(
#         128,
#         256,
#         3,
#         stride=2,
#         padding=1,
#         )  
        #现在x1变成512*256*125(125是指250+1中含有的奇数数量减去1)的了
        x1 = self._conv3_1(x1)
#         self._conv3_1 = Conv1D(
#         256,
#         512,
#         3,
#         stride=2,
#         padding=1,
#         )
        
        # 现在x1变成512*512*63（63等于125+1后里面含有的奇数数量为63个，这里貌似我不清楚他遇到偶数是怎么处理的，跟我想的
        # 好像有点不一样，卷积核的屁股遇到第63个奇数125之后，往后移动移动不了了，因为后面只能再推一格，而步长是2推不动
        # 此时只有62个数，可能他会继续往后推到底再搞一个数，才是63个吧 不过这里感觉影响不大）
        
        x1 = F.relu(x1)
        #现在x1应该还是512*512*63，应该是对每一个小格子来一波relu吧
        x1 = self._conv4_1(x1)
#         self._conv4_1 = Conv1D(
#         512,
#         256,
#         3,
#         stride=2,
#         padding=1,
#         )
        # 512*512*63变成512*256*32（又遇到偶数的情况了，就默认是63+1除以2吧）
        x1 = self._conv5_1(x1)
        x1 = F.relu(x1)
        x1 = self._conv6_1(x1)
        x1 = F.relu(x1)

        x2 = self._conv2_2(x2)
        x2 = self._conv3_2(x2)
        x2 = F.relu(x2)
        x2 = self._conv4_2(x2)
        x2 = self._conv5_2(x2)         
        x2 = F.relu(x2)
        x2 = self._conv6_2(x2)
        x2 = F.relu(x2)  
        
        
        x = paddle.concat(x = [x1,x2], axis=2)
        x = paddle.flatten(x, start_axis=1, stop_axis=-1)
        x = self._fc8(x)
        return x

### 网络搭建与可视化-原版网络

In [4]:
import math
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
import pywt
from paddle.nn import Linear, Dropout, ReLU
from paddle.nn import Conv1D, MaxPool1D
from paddle.nn.initializer import Uniform
from paddle.fluid.param_attr import ParamAttr
from paddle.utils.download import get_weights_path_from_url

class MyNet_dwt(nn.Layer):

    def __init__(self, num_classes=1000):
        super(MyNet_dwt, self).__init__()
        self.num_classes = num_classes
        
        self._conv1 = Conv1D(
            65,
            128,
            3,
            stride=2,
            padding=1,
            )      

        self._conv2_1 = Conv1D(
            128,
            256,
            3,
            stride=2,
            padding=1,
            )  
        self._conv3_1 = Conv1D(
            256,
            512,
            3,
            stride=2,
            padding=1,
            )
        self._conv4_1 = Conv1D(
            512,
            256,
            3,
            stride=2,
            padding=1,
            )

        self._conv2_2 = Conv1D(
            128,
            256,
            3,
            stride=2,
            padding=1,
            )  
        self._conv3_2 = Conv1D(
            256,
            512,
            3,
            stride=2,
            padding=1,
            )
        
        self._conv4_2 = Conv1D(
            512,
            256,
            3,
            stride=2,
            padding=1,
            )

        self._fc8 = Linear(
            in_features=16384,
            out_features=num_classes,
            )

    def forward(self, inputs):
        x = self._conv1(inputs)
        x = paddle.to_tensor(pywt.dwt(x.numpy(), 'haar'), dtype='float32')
        x1,x2 = x.split(2)

        x1 = x1.squeeze(axis=0)
        x2 = x2.squeeze(axis=0)
        x1 = self._conv2_1(x1)
        x1 = self._conv3_1(x1)
        x1 = F.relu(x1)
        x1 = self._conv4_1(x1)
        x1 = F.relu(x1)

        x2 = self._conv2_2(x2)
        x2 = self._conv3_2(x2)
        x2 = F.relu(x2)
        x2 = self._conv4_2(x2)
        x2 = F.relu(x2)      
        
        x = paddle.concat(x = [x1,x2], axis=2)
        x = paddle.flatten(x, start_axis=1, stop_axis=-1)
        x = self._fc8(x)
        return x

In [5]:
model_res = MyNet_dwt(num_classes=95)
paddle.summary(model_res,(512,65,1000))# input shape 512应该是batch size 咱们还是别管他了

W0521 01:23:11.181232 34569 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 8.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0521 01:23:11.182246 34569 dynamic_loader.cc:307] The third-party dynamic library (libcudnn.so) that Paddle depends on is not configured correctly. (error code is /usr/local/cuda/lib64/libcudnn.so: cannot open shared object file: No such file or directory)
  Suggestions:
  1. Check if the third-party dynamic library (e.g. CUDA, CUDNN) is installed correctly and its version is matched with paddlepaddle you installed.
  2. Configure third-party dynamic library environment variables as follows:
  - Linux: set LD_LIBRARY_PATH by `export LD_LIBRARY_PATH=...`
  - Windows: set PATH by `set PATH=XXX;


RuntimeError: (PreconditionNotMet) Cannot load cudnn shared library. Cannot invoke method cudnnGetVersion.
  [Hint: cudnn_dso_handle should not be null.] (at /paddle/paddle/phi/backends/dynload/cudnn.cc:60)


### 3.4 模型配置与训练

In [ ]:
# 模型封装
model = paddle.Model(model_res)

# 定义优化器
class Cosine(CosineAnnealingDecay):
    """
    Cosine learning rate decay
    lr = 0.05 * (math.cos(epoch * (math.pi / epochs)) + 1)

    Args:
        lr(float): initial learning rate
        step_each_epoch(int): steps each epoch
        epochs(int): total training epochs
    """

    def __init__(self, lr, step_each_epoch, epochs, **kwargs):
        super(Cosine, self).__init__(
            learning_rate=lr,
            T_max=step_each_epoch * epochs, )

        self.update_specified = False

class CosineWarmup(LinearWarmup):
    """
    Cosine learning rate decay with warmup
    [0, warmup_epoch): linear warmup
    [warmup_epoch, epochs): cosine decay

    Args:
        lr(float): initial learning rate
        step_each_epoch(int): steps each epoch
        epochs(int): total training epochs
        warmup_epoch(int): epoch num of warmup
    """

    def __init__(self, lr, step_each_epoch, epochs, warmup_epoch=5, **kwargs):
        assert epochs > warmup_epoch, "total epoch({}) should be larger than warmup_epoch({}) in CosineWarmup.".format(
            epochs, warmup_epoch)
        warmup_step = warmup_epoch * step_each_epoch
        start_lr = 0.0
        end_lr = lr
        lr_sch = Cosine(lr, step_each_epoch, epochs - warmup_epoch)

        super(CosineWarmup, self).__init__(
            learning_rate=lr_sch,
            warmup_steps=warmup_step,
            start_lr=start_lr,
            end_lr=end_lr)

        self.update_specified = False

scheduler = CosineWarmup(
        lr=0.00125, step_each_epoch=226, epochs=24, warmup_steps=20, start_lr=0, end_lr=0.00125, verbose=True)
optim = paddle.optimizer.Adam(learning_rate=scheduler, parameters=model.parameters())

# 配置模型
model.prepare(
    optim,
    paddle.nn.CrossEntropyLoss(),
    Accuracy()
    )
callback = paddle.callbacks.VisualDL(log_dir='visualdl_log_dir_alexdwt')


# 模型训练与评估
model.fit(train_loader,
        val_loader,
        log_freq=1,
        epochs=24,
        callbacks=callback,
        verbose=1,
        )

In [ ]:
model.save('Hapi_MyCNN_dwt', True)  # save 

### 模型预测与结果提交

In [ ]:
# 模型预测并生成提交文件

import os, time
import matplotlib.pyplot as plt
import paddle
from PIL import Image
import numpy as np
import pandas as pd
import scipy.io as scio

use_gpu = True
# use_gpu = False
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

param_state_dict = paddle.load( "Hapi_MyCNN_dwt.pdparams")
model_res.set_dict(param_state_dict)
model_res.eval() #训练模式

test_image = pd.read_csv('/home/u200810216/jupyter/kaggle_m3cv/Testing_Info.csv')
test_image_path_list = test_image['EpochID'].values

eeg_list = list()

labeled_img_list = []
for img in test_image_path_list:
    eeg_list.append('/home/u200810216/jupyter/kaggle_m3cv/test/'+img+'.mat')
    labeled_img_list.append(img)

def load_eeg(eeg_path):
    # 读取数据
    data = scio.loadmat(eeg_path)
    return data['epoch_data']

pre_list = []
for i in range(len(eeg_list)):
    data = load_eeg(eeg_path=eeg_list[i])
    dy_x_data = np.array(data).astype('float32')
    dy_x_data = dy_x_data[np.newaxis,:, :]
    # 即使是一个测试的数据，也要搞成数据集的形式，这为下面那个paddle.nn.functional.softmax(out)[0]中的[0]作下了铺垫。
    eeg = paddle.to_tensor(dy_x_data)
    out = model_res(eeg)
    res = paddle.nn.functional.softmax(out)[0] # 若模型中已经包含softmax则不用此行代码。这个0代表所得到列表中的第一个元素
    
    lab = np.argmax(out.numpy())  #argmax():返回最大数的索引
    pre_list.append(int(lab)+1)
    
img_test = pd.DataFrame(labeled_img_list)
img_pre = pd.DataFrame(labeled_img_list)

img_test = img_test.rename(columns = {0:"EpochID"})
img_pre['SubjectID'] = pre_list
pre_info = img_pre['SubjectID'].values
test_info = test_image['SubjectID'].values

result_cnn = list()

for i,j in zip(test_info, pre_info):

    if i == 'None':
        result_cnn.append(j)
    elif int(i[4:])==j :
        print(i[4:])
        result_cnn.append(int(1))
    else:
        result_cnn.append(int(0))

img_test['Prediction'] = result_cnn
img_test.to_csv('result_dwt.csv', index=False)

## 总结

在本项目中通过提出的网络结构，完成了模型训练、结果提交全流程，包括数据准备、模型训练及保存和预测步骤，并完成了预测结果提交。对于模型的调优，大家可以从以下几个方面考虑：

1、对数据集进行预处理，使用Label Smooth对标签进行处理

2、模型的选择，增加网络深度，添加残差结构等

3、模型的超参数，如学习率、batch_size等参数

4、模型训练方式，包括优化器的选择、WarmUp方式选择



如果项目中有任何问题，欢迎在评论区留言交流，共同进步！

## 以下为草稿

In [1]:
import paddle
paddle.utils.run_check()

Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle works well on 1 GPUs.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


In [10]:
import paddle
from paddle.nn import Conv1D
import numpy as np
x = np.array([[[4, 8, 1, 9],
  [7, 2, 0, 9],
  [6, 9, 2, 6]]]).astype(np.float32)

# 这里的w就是那两个kernel怎么定义! 你看下面那个Conv1D(3, 2, 3)中间的2就是两个output channel 就是2个filter的意思。
# 扫的时候就对应相乘，所以不同的filter的差别，在于是权重上的不同！！！！ 本身不同filter自己就不一样！
# 此外我觉得那个filter的大小 3 的意思，应该不是说必须是3*3的矩阵大小，这里横着有三行应该是继承了input channel的数量。
w=np.array(
[[[9, 3, 4],
  [0, 0, 7],
  [2, 5, 6]],
 [[0, 3, 4],
  [2, 9, 7],
  [5, 6, 8]]]).astype(np.float32)
x_t = paddle.to_tensor(x)
conv = Conv1D(3, 2, 3)
conv.weight.set_value(w)


y_t = conv(x_t)
print(y_t)
# [[[133. 238.]
#   [160. 211.]]]

# 下面的实验说明卷积核的大小是由input channel和kernel size共同决定的！
x = np.array([[
    [4, 8, 1, 9],
    [7, 2, 0, 9]
              ]]).astype(np.float32)
w=np.array(
[[[9, 3, 4],
  [0, 0, 7]],
 [[0, 3, 4],
  [2, 9, 7]]]).astype(np.float32)
x_t = paddle.to_tensor(x)
conv = Conv1D(2, 2, 3)
conv.weight.set_value(w)

y_t = conv(x_t)
print(y_t)
# [[[133. 238.]
#   [160. 211.]]]

Tensor(shape=[1, 2, 2], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[[133., 238.],
         [160., 211.]]])
Tensor(shape=[1, 2, 2], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[[64. , 174.],
         [60. , 106.]]])
